TP1 IBD - consultas sobre Banco de Dados com dados de Fórmula 1, dos anos de 1950 a 2018

In [1]:
#nova conexão e importação dos dados do Banco de Dados
import io
import sqlite3
import pandas as pd

In [2]:
#criação do BD a partir do arquivo database.sql
nomeArquivoBanco = "BDtp1.db"

# Conecta (ou cria) o banco de dados
conn = sqlite3.connect(nomeArquivoBanco)
cursor = conn.cursor()

# Lê o conteúdo do arquivo .sql
with open("database.sql", "r", encoding="utf-8") as arquivo_sql:
    script_sql = arquivo_sql.read()

# Executa os comandos SQL contidos no script
cursor.executescript(script_sql)

# Salva as alterações e fecha a conexão
conn.commit()

In [ ]:
#onsulta A0
query = """
SELECT NOME_PAIS, NOME_CIRCUITO
FROM PAIS 
NATURAL JOIN
(SELECT ID_CIRCUITO, NOME_CIRCUITO, ANO_CORRIDA
 FROM CIRCUITO
 NATURAL JOIN CORRIDA
 WHERE CORRIDA.ANO_CORRIDA = 1987) 
"""

df = pd.read_sql_query(query, conn)
df

,NOME_PAIS,NOME_CIRCUITO
0,Alemanha,Circuit de Monaco
1,Alemanha,Silverstone Circuit
2,Alemanha,Hockenheimring
3,Alemanha,Hungaroring
4,Alemanha,Circuit de Spa-Francorchamps
...,...,...
507,Índia,Adelaide Street Circuit
508,Índia,Autódromo Hermanos Rodríguez
509,Índia,Circuit Paul Ricard
510,Índia,Autódromo Internacional Nelson Piquet


In [ ]:
#consulta A1
query = """
SELECT NOME_CONSTRUTOR
FROM CONSTRUTOR
NATURAL JOIN
(SELECT ID_CONSTRUTOR
 FROM RESULTADO
 NATURAL JOIN STATUS
 WHERE STATUS.DESC_STATUS = 'Velas' OR STATUS.DESC_STATUS = 'Turbo')
"""
df = pd.read_sql_query(query, conn)
df

,NOME_CONSTRUTOR
0,Arrows
1,Zakspeed
2,Benetton
3,Benetton
4,Minardi
...,...
141,Kurtis Kraft
142,Red Bull
143,Caterham
144,McLaren


In [ ]:
#consulta A2
#usamos SELECT DISTINCT porque há pilotos que o fizeram em várias corridas, gerando "duplicatas" na saída
query = """
SELECT DISTINCT NOME_PILOTO, NACIONALIDADE_PILOTO
FROM PILOTO
NATURAL JOIN RESULTADO as R
WHERE R.VALOR_POS_TERMINO = 1 AND VALOR_POS_LARGADA = 1 AND VALOR_POSICAO_VMR = 1
"""
df = pd.read_sql_query(query, conn)
df

,NOME_PILOTO,NACIONALIDADE_PILOTO
0,Kimi Räikkönen,Finlandesa
1,Felipe Massa,Brasileira
2,Lewis Hamilton,Britânica
3,Fernando Alonso,Espanhola
4,Michael Schumacher,Alemã
5,Rubens Barrichello,Brasileira
6,Jenson Button,Britânica
7,Sebastian Vettel,Alemã
8,Nico Rosberg,Alemã
9,Valtteri Bottas,Finlandesa


In [18]:
#consulta A3
query = """
SELECT NOME_PAIS, NOME_CIRCUITO
FROM PAIS as P
LEFT OUTER JOIN CIRCUITO as C ON P.ID_PAIS = C.ID_PAIS
"""
df = pd.read_sql_query(query, conn)
df

,NOME_PAIS,NOME_CIRCUITO
0,Alemanha,AVUS
1,Alemanha,Hockenheimring
2,Alemanha,Nürburgring
3,Argentina,Autódromo Oscar y Juan Gálvez
4,Austrália,Adelaide Street Circuit
...,...,...
68,África do Sul,Prince George Circuit
69,Áustria,A1-Ring
70,Áustria,Red Bull Ring
71,Áustria,Zeltweg


In [25]:
#consulta A4
query = """
SELECT NACIONALIDADE_PILOTO
FROM PILOTO

INTERSECT 

SELECT DISTINCT NACIONALIDADE_CONSTRUTOR
FROM CONSTRUTOR
"""
df = pd.read_sql_query(query, conn)
df

,NACIONALIDADE_PILOTO
0,Alemã
1,Alemã oriental
2,Americana
3,Australiana
4,Austríaca
5,Belga
6,Brasileira
7,Britânica
8,Canadense
9,Espanhola


In [29]:
#consulta B0
query = """
SELECT NOME_CIRCUITO, NOME_PILOTO
FROM PILOTO
NATURAL JOIN
(SELECT NOME_CIRCUITO, ID_CORRIDA
 FROM CIRCUITO
 NATURAL JOIN CORRIDA
 WHERE ANO_CORRIDA = 2017)
NATURAL JOIN RESULTADO
WHERE VALOR_POS_TERMINO = 1
"""
df = pd.read_sql_query(query, conn)
df

,NOME_CIRCUITO,NOME_PILOTO
0,Albert Park Grand Prix Circuit,Sebastian Vettel
1,Shanghai International Circuit,Lewis Hamilton
2,Bahrain International Circuit,Sebastian Vettel
3,Sochi Autodrom,Valtteri Bottas
4,Circuit de Barcelona-Catalunya,Lewis Hamilton
5,Circuit de Monaco,Sebastian Vettel
6,Circuit Gilles Villeneuve,Lewis Hamilton
7,Baku City Circuit,Daniel Ricciardo
8,Red Bull Ring,Valtteri Bottas
9,Silverstone Circuit,Lewis Hamilton


In [36]:
#consulta B1
query = """
SELECT P.NOME_PILOTO, P.NACIONALIDADE_PILOTO, COUNT(*) AS NRO_POLE_POS
FROM CORRIDA AS C
JOIN RESULTADO AS R ON C.ID_CORRIDA = R.ID_CORRIDA
JOIN PILOTO AS P ON P.ID_PILOTO = R.ID_PILOTO
WHERE R.VALOR_POS_LARGADA = 1 AND C.ANO_CORRIDA = 2015 
GROUP BY P.ID_PILOTO
ORDER BY NRO_POLE_POS DESC
LIMIT 1
"""
df = pd.read_sql_query(query, conn)
df

,NOME_PILOTO,NACIONALIDADE_PILOTO,NRO_POLE_POS
0,Lewis Hamilton,Britânica,11


In [39]:
#consulta B2
query = """
SELECT C.NOME_CONSTRUTOR, P.DURACAO_PITSOP
FROM PITSTOP AS P
JOIN RESULTADO AS R ON R.ID_CORRIDA = P.ID_CORRIDA
JOIN CONSTRUTOR AS C ON C.ID_CONSTRUTOR = R.ID_CONSTRUTOR
ORDER BY P.DURACAO_PITSOP ASC 
LIMIT 1
"""
df = pd.read_sql_query(query, conn)
df

,NOME_CONSTRUTOR,DURACAO_PITSOP
0,McLaren,12.897


In [53]:
#consulta B5
query = """
SELECT DISTINCT P.NOME_PAIS
FROM PAIS AS P
NATURAL JOIN CIRCUITO AS CI
NATURAL JOIN CORRIDA AS C
GROUP BY C.ANO_CORRIDA, P.ID_PAIS
HAVING COUNT(*) > 1
"""
df = pd.read_sql_query(query, conn)
df

,NOME_PAIS
0,Itália
1,EUA
2,França
3,Reino Unido
4,Alemanha
5,Espanha
6,Japão


In [60]:
#consulta B9
query = """
SELECT CI.NOME_CIRCUITO, COUNT(*) AS QT_VITORIAS
FROM CORRIDA AS C
JOIN CIRCUITO AS CI ON CI.ID_CIRCUITO = C.ID_CIRCUITO
JOIN RESULTADO AS R ON R.ID_CORRIDA = C.ID_CORRIDA
JOIN PILOTO AS P ON R.ID_PILOTO = P.ID_PILOTO
WHERE P.NOME_PILOTO = 'Lewis Hamilton' AND R.VALOR_POS_TERMINO = 1
GROUP BY CI.NOME_CIRCUITO
ORDER BY QT_VITORIAS DESC
"""
df = pd.read_sql_query(query, conn)
df

,NOME_CIRCUITO,QT_VITORIAS
0,Circuit Gilles Villeneuve,6
1,Silverstone Circuit,5
2,Shanghai International Circuit,5
3,Hungaroring,5
4,Circuit of the Americas,5
5,Autodromo Nazionale di Monza,4
6,Yas Marina Circuit,3
7,Suzuka Circuit,3
8,Marina Bay Street Circuit,3
9,Circuit de Spa-Francorchamps,3
